### Logistic Regression - Label Classification

In [22]:
import sys
sys.path.append('../../../../')

# Import libraries
from utils.score import fold_cross_validate
from utils.print import print_cross_validation_scores
from constant.columns import FEATURES, LABEL

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [23]:
# Read dataset
df = pd.read_csv(f'./source/dataset_smote.csv')
X, y = df[FEATURES], df[LABEL]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

#### Cross Validation

In [24]:
# Define the model
clf_cv = LogisticRegression()
scores = fold_cross_validate(clf_cv, X_test, y_test)

# Print scores
print_cross_validation_scores(scores)

MACRO:
Precision: 0.7963 (0.0353)
Recall: 0.7933 (0.0358)
F1 score: 0.7910 (0.0372)
---------------------------------
MICRO:
Precision: 0.7922 (0.0375)
Recall: 0.7922 (0.0375)
F1 score: 0.7922 (0.0375)
---------------------------------


In [25]:
# Define the optimized model (previous research)
clf_cv = LogisticRegression(penalty="l1", C=1, solver="liblinear", max_iter=100, class_weight=None, fit_intercept=False, multi_class="auto")
scores = fold_cross_validate(clf_cv, X_test, y_test)

# Print scores
print_cross_validation_scores(scores)

MACRO:
Precision: 0.8082 (0.0253)
Recall: 0.8058 (0.0258)
F1 score: 0.8039 (0.0270)
---------------------------------
MICRO:
Precision: 0.8050 (0.0272)
Recall: 0.8050 (0.0272)
F1 score: 0.8050 (0.0272)
---------------------------------
